# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [82]:
import selenium

In [83]:
from selenium import webdriver

In [84]:
driver = webdriver.Chrome()

In [85]:
driver.get("https://jportal.mdcourts.gov/license/index_disclaimer.jsp")

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [86]:
box = driver.find_element_by_xpath('//*[@id="checkbox"]')
box.click()

In [87]:
enter_site = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_site.click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [88]:
search_records = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[6]/td[2]/a[2]')
search_records.click()

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [89]:
juri = driver.find_element_by_xpath('//*[@id="slcJurisdiction"]/option[2]')
juri.click()
#select = Select(juri)
# select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [90]:
trade_name = driver.find_element_by_xpath('//*[@id="txtTradeName"]')
trade_name.click()

In [91]:
trade_name.send_keys("vap%")

In [92]:
# count = 0
# while True:
#  try:
#  Hey, to get that link and click it
# driver.get_the_thing("Next").click()
# except:
#  Oh did you get an error? Exit the while loop 
# break 

### How do you click the submit button or submit the form?

In [93]:
submit = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit.click()

### How can you find and click the 'Next' button on the search results page?

In [94]:
next_but = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
next_but.click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [95]:
results = []
while True:
    doc = BeautifulSoup(driver.page_source, 'html.parser')
    business_headers = doc.find_all('tr',class_='searchfieldtitle')
    for header in business_headers:
        rows = header.find_next_siblings('tr')
        current = {}
        current['Firm'] = rows[0].text.strip()
        current['Address'] = rows[1].text.strip()
        current['Area'] = rows[2].text.strip()
        current['County'] = rows[3].text.strip()
        results.append(current)
        print(current)
    try:
        next_but = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr/td[3]/a/nobr')
        next_but.click()
    except:
        break

{'Firm': 'DISBROW II EMERSON HARRINGTON\nLic. Status: Issued', 'Address': '185 MITCHELLS CHANCE RD\nLicense: 02102408', 'Area': 'EDGEWATER, MD 21037\nIssued Date: 4/13/2017', 'County': 'Anne Arundel County'}
{'Firm': 'DISCOUNT TOBACCO ESSEX LLC\nLic. Status: Issued', 'Address': '7104 MINSTREL UNIT #7\nLicense: 13141786', 'Area': 'COLUMBIA, MD 21045\nIssued Date: 5/19/2017', 'County': 'Howard County'}
{'Firm': 'FAIRGROUND VILLAGE LLC\nLic. Status: Issued', 'Address': '330 ONE FORTY VILLAGE ROAD\nLicense: 06126253', 'Area': 'WESTMINSTER, MD 21157\nIssued Date: 4/21/2017', 'County': 'Carroll County'}
{'Firm': 'GRIMM JENNIFER\nLic. Status: Pending', 'Address': '29890 THREE NOTCH ROAD', 'Area': 'CHARLOTTE HALL, MD 20622', 'County': "St. Mary's County"}
{'Firm': 'HUTCH VAPES LLC\nLic. Status: Issued', 'Address': '356 ROMANCOKE ROAD\nLicense: 17166688', 'Area': 'STEVENSVILLE, MD 21666\nIssued Date: 4/13/2017', 'County': "Queen Anne's County"}
{'Firm': 'HUTCHINS COX LLP\nLic. Status: Pending',

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [96]:
import requests
from bs4 import BeautifulSoup
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [97]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.
business_headers = doc.find_all('tr',class_='searchfieldtitle')
len(business_headers)

2

### Save these into `vape-results.csv`

In [98]:
import pandas as pd

In [99]:
df = pd.DataFrame(results)
df.head()

,Address,Area,County,Firm
0,185 MITCHELLS CHANCE RD\nLicense: 02102408,"EDGEWATER, MD 21037\nIssued Date: 4/13/2017",Anne Arundel County,DISBROW II EMERSON HARRINGTON\nLic. Status: Is...
1,7104 MINSTREL UNIT #7\nLicense: 13141786,"COLUMBIA, MD 21045\nIssued Date: 5/19/2017",Howard County,DISCOUNT TOBACCO ESSEX LLC\nLic. Status: Issued
2,330 ONE FORTY VILLAGE ROAD\nLicense: 06126253,"WESTMINSTER, MD 21157\nIssued Date: 4/21/2017",Carroll County,FAIRGROUND VILLAGE LLC\nLic. Status: Issued
3,29890 THREE NOTCH ROAD,"CHARLOTTE HALL, MD 20622",St. Mary's County,GRIMM JENNIFER\nLic. Status: Pending
4,356 ROMANCOKE ROAD\nLicense: 17166688,"STEVENSVILLE, MD 21666\nIssued Date: 4/13/2017",Queen Anne's County,HUTCH VAPES LLC\nLic. Status: Issued


In [100]:
df.to_csv("vape-results.csv", index=False)

### Open `vape-results.csv` to make sure there aren't any extra weird columns

In [101]:
results_df = pd.read_csv("vape-results.csv")
results_df.head()

,Address,Area,County,Firm
0,185 MITCHELLS CHANCE RD\nLicense: 02102408,"EDGEWATER, MD 21037\nIssued Date: 4/13/2017",Anne Arundel County,DISBROW II EMERSON HARRINGTON\nLic. Status: Is...
1,7104 MINSTREL UNIT #7\nLicense: 13141786,"COLUMBIA, MD 21045\nIssued Date: 5/19/2017",Howard County,DISCOUNT TOBACCO ESSEX LLC\nLic. Status: Issued
2,330 ONE FORTY VILLAGE ROAD\nLicense: 06126253,"WESTMINSTER, MD 21157\nIssued Date: 4/21/2017",Carroll County,FAIRGROUND VILLAGE LLC\nLic. Status: Issued
3,29890 THREE NOTCH ROAD,"CHARLOTTE HALL, MD 20622",St. Mary's County,GRIMM JENNIFER\nLic. Status: Pending
4,356 ROMANCOKE ROAD\nLicense: 17166688,"STEVENSVILLE, MD 21666\nIssued Date: 4/13/2017",Queen Anne's County,HUTCH VAPES LLC\nLic. Status: Issued


## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.